# KEN3450, Data Analysis 2020 

**Kaggle Competition 2020**<br>

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import csv

#import your classifiers here

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings(action='once')

# Diagnosing the Maastricht Flu 

You are given the early data for an outbreak of a dangerous virus originating from a group of primates being kept in a Maastricht biomedical research lab in the basement of Henri-Paul Spaaklaan building, this virus is dubbed the "Maastricht Flu".

You have the medical records of $n$ number of patients in `flu_train.csv`. There are two general types of patients in the data, flu patients and healthy (this is recorded in the column labeled `flu`, a 0 indicates the absences of the virus and a 1 indicates presence). Notice that the dataset is unbalanced and you can expect a similar imbalance in the testing set.

**Your task:** build a model to predict if a given patient has the flu. Your goal is to catch as many flu patients as possible without misdiagnosing too many healthy patients.

**The deliverable:** submit your final solution via Kaggle competition using the `flu_test.csv` data.

Maastricht Gemeente will use your model to diagnose sets of future patients (held by us). You can expect that there will be an increase in the number of flu patients in any groups of patients in the future.

Here are some benchmarks for comparison and for expectation management. Notice that because the dataset is unbalanced, we expect that there is going to be a large difference in the accuracy for each class, thus `accuracy` is a metric that might be misleading in this case (see also below). That's why the baselines below are based on the expected accuracy **per class** and also they give you an estimate for the AUROC on all patients in the testing data. This is the score you see in the Kaggle submission as well.

**Baseline Model:** 
- ~50% expected accuracy on healthy patients in training data
- ~50% expected accuracy on flu patients in training data
- ~50% expected accuracy on healthy patients in testing data (future data, no info on the labels)
- ~50% expected accuracy on flu patients in testing data (future data, no info on the labels)
- ~50% expected AUROC on all patients in testing data (future data, no info on the labels)

**Reasonable Model:** 
- ~70% expected accuracy on healthy patients in training data
- ~55% expected accuracy on flu patients, in training data
- ~70% expected accuracy on healthy patients in testing data (future data, no info on the labels, to be checked upon your submission)
- ~57% expected accuracy on flu patients, in testing data (future data, no info on the labels, to be checked upon your submission)
- ~65% expected AUROC on all patients, in testing data (future data, no info on the labels, to be checked from Kaggle)

**Grading:**
Your grade will be based on:
1. your model's ability to out-perform the benchmarks (they are kind of low, so we won't care much about this)
2. your ability to carefully and thoroughly follow the data analysis pipeline
3. the extend to which all choices are reasonable and defensible by methods you have learned in this class

## Step 1: Read the data, clean and explore the data

There are a large number of missing values in the data. Nearly all predictors have some degree of missingness. Not all missingness are alike: NaN in the `'pregnancy'` column is meaningful and informative, as patients with NaN's in the pregnancy column are males, where as NaN's in other predictors may appear randomly. 


**What do you do?:** We make no attempt to interpret the predictors and we make no attempt to model the missing values in the data in any meaningful way. We replace all missing values with 0.

However, it would be more complete to look at the data and allow the data to inform your decision on how to address missingness. For columns where NaN values are informative, you might want to treat NaN as a distinct value; You might want to drop predictors with too many missing values and impute the ones with few missing values using a model. There are many acceptable strategies here, as long as the appropriateness of the method in the context of the task and the data is discussed.

In [ ]:
#Train
df = pd.read_csv('data/flu_train.csv')
df = df[~np.isnan(df['flu'])]
df.head()

In [ ]:
#Test
df_test = pd.read_csv('data/flu_test.csv')
df_test.head()

In [ ]:
#What's up in each set

x = df.values[:, :-1]
y = df.values[:, -1]

x_test = df_test.values[:, :-1]

print('x train shape:', x.shape)
print('x test shape:', x_test.shape)
print('train class 0: {}, train class 1: {}'.format(len(y[y==0]), len(y[y==1])))

In [ ]:
# Lets count the amount of Nan
nan_count = df.isna().sum()
nan_percentage = [count / x.shape[0] for count in nan_count]
columns = df.columns

threshold = 0.6

# And let's plot those
fig1 = plt.figure(figsize=(30,15))
ax = fig1.add_subplot(111)
plt.axhline(y=threshold,linewidth=1, color="k")
ax.bar(columns, nan_percentage)
plt.xticks(rotation=90)
ax.set_xlabel('Column')
ax.set_ylabel('NaN Count')
ax.set_title('Columns with missing values')

We can see that there are many columns that have missing values. Of course it is not wise to just blatantly drop every observation that has a NaN value somewhere, however some columns have so many missing values, some over 75% of the data! For now we will drop these columns because they won't be able to indicate sickness if they don't have information about the majority of the dataset

In [ ]:
too_much_cols = []
for i in range(len(columns)):
    if nan_percentage[i] >= threshold:
        too_much_cols.append(columns[i])
df_cut = df.drop(too_much_cols, axis=1)
print("So we went from {} features to {} features, based on too much NaN".format(df.shape[1], df_cut.shape[1]))

In [ ]:
# Do another count
nan_count = df_cut.isna().sum()
nan_percentage = [count / x.shape[0] for count in nan_count]
columns = df_cut.columns

# And let's plot those
fig1 = plt.figure(figsize=(30,15))
ax = fig1.add_subplot(111)
plt.axhline(y=threshold,linewidth=1, color="k")
ax.bar(columns, nan_percentage)
plt.xticks(rotation=90)
ax.set_xlabel('Column')
ax.set_ylabel('NaN Count')
ax.set_title('Columns with missing values')

In [ ]:
df.describe()

Now let's have a look at the distribution of the dataset

In [ ]:
sick_df = df_cut[(df['flu'] == 1)]
okay_df = df_cut[(df['flu'] == 0)]

plt.bar(['Sick', 'Healthy'], [sick_df.shape[0], okay_df.shape[0]])
plt.show()
print("So we have {}% sick people and {}% healthy people".format(np.round((sick_df.shape[0]/df_cut.shape[0])*100, 
                                                                          decimals=2), 
                                                                 np.round((okay_df.shape[0]/df_cut.shape[0])*100, 
                                                                          decimals=2)))

In [ ]:
df_cut.columns

Now lets see what values our columns have so we can try to quantify those for our model

In [ ]:
for column in df_cut.select_dtypes(include='object').columns:
    print("{} has values:\t\t\t{}\n".format(column, df_cut[column].unique()))

Nothing weird to see here, columns with simple 'male'/'female' or 'yes'/'no' can easily be changed to numerical values. Some more categorical columns need some more thought on how to feed those to our model

In [ ]:
#df_cut['Gender'].replace(['female','male'],[0,1],inplace=True)

dummies = ['Race1', 
           'Education', 
           'MaritalStatus', 
           'HomeOwn', 
           'BMI_WHO', 
           'HealthGen', 
           'LittleInterest', 
           'Depressed', 
           'TVHrsDay', 
           'CompHrsDay', 
           'SexOrientation'] 

for column in dummies:
    df_cut = pd.concat([df_cut, pd.get_dummies(df_cut[column])], axis=1)
    
df_cut.drop(dummies, axis=1)
    
print(df_cut.columns)

# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['Education'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['MaritalStatus'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['HomeOwn'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['BMI_WHO'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['HealthGen'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['LittleInterest'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['Depressed'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['TVHrsDay'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['CompHrsDay'])], axis=1)
# df_cut = pd.concat([df_cut, df_cut.get_dummies(df_cut['SexOrientation'])], axis=1)

df_cut

In [ ]:
yes_no_cols = ['HardDrugs','Diabetes', 'SleepTrouble', 'PhysActive', 'Alcohol12PlusYr', 
              'Smoke100', 'Smoke100n', 'Marijuana', 'RegularMarij', 'HardDrugs', 'SexEver', 'SameSex']
yes_vals = []
no_vals = []

for column in yes_no_cols:
    yes_vals.append(df_cut[df_cut[column] == 'Yes']['flu'].sum())
    no_vals.append(df_cut[df_cut[column] == 'No']['flu'].sum())

ind = np.arange(len(yes_no_cols))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

plt.figure(figsize=(10,5))
p1 = plt.bar(ind, no_vals, width)
p2 = plt.bar(ind, yes_vals, width,
             bottom=no_vals)
plt.ylabel('Amount')
plt.title('Ratio between \'Yes\' and \'No\'')

plt.xticks(ind, (yes_no_cols), rotation=50, ha='right')

plt.legend((p1[0], p2[0]), ('No', 'Yes'))
plt.show()

In [ ]:
# LinReg for unclear difference, else just choose the dominant answer for NaN values in columns, where it suggests a sick person
# LinReg might create some bias/ some relation that is coincidental
# Dominant answer for sick people will yield some false positives, preferable over sick people classified as healthy
df_cut['HardDrugs'].fillna('No')
df_cut['HardDrugs'].replace(['No','Yes'],[0,1],inplace=True)

df_cut['Diabetes'].fillna('No')
df_cut['Diabetes'].replace(['No','Yes'],[0,1],inplace=True)

df_cut['Alcohol12PlusYr'].fillna('Yes')
df_cut['Alcohol12PlusYr'].replace(['No','Yes'],[0,1],inplace=True)

df_cut['SexEver'].fillna('Yes')
df_cut['SexEver'].replace(['No','Yes'],[0,1],inplace=True)

df_cut['SameSex'].fillna('No')
df_cut['SameSex'].replace(['No','Yes'],[0,1],inplace=True)

In [ ]:
for column in df_cut.select_dtypes(include='float64').columns:
    print("{} has range\t\t\t[{}, {}]".format(column, df_cut[column].min(), df_cut[column].max()))

In [ ]:
df_num_only = df_cut[df_cut.select_dtypes(exclude='object').columns]
used_columns = df_cut.select_dtypes(exclude='object').columns
print(used_columns[:-1])
df_num_only.fillna(0, inplace=True)
X = np.array(df_num_only.loc[:, df_num_only.columns != 'flu'])
y = np.array(df_num_only['flu']).reshape(-1,1)
print(X)
print(y)

Now this is where we find some weird ranges in our data.
* Weight has a range of 2.8 to 223 (Babies v.s. obese?)
* Height has a range of 83.6 to 200.4 (Babies v.s. Basketballers?)
* Alcohol per Day has highest value of 82 ??
* SexNumPartnLife probably has highest value of 2000...

In [ ]:
from sklearn.model_selection import train_test_split
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y.ravel(), test_size=0.3)


In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(X_train_0, y_train_0.ravel())
print(X_train_0.shape)
print(y_train_0.shape)
print(X_resampled.shape)
print(y_resampled.shape)

print(y_train_0.sum())
print(y_resampled.sum())

In [ ]:
sick_df = y.sum()
okay_df = y_resampled.sum()

plt.bar(['Sick before','Healthy before', 'Sick after', 'Healthy'], 
        [y.sum(),(len(y)-y.sum()),
        y_resampled.sum(), (len(y_resampled)-y_resampled.sum())],
        color=['purple', 'green', 'purple','green'])
plt.show()

## Step 2: Model Choice

The first task is to decide which classifier to use (from the ones that we learned this block), i.e. which one would best suit our task and our data. Note that our data are heavily unbalanced, thus you need to do some exploration on how different classifiers handle inbalances in the data (we will discuss some of these techniques during week 3 lecture).

It would be possible to do brute force model comparison here - i.e. tune all models and compare which does best with respect to various benchmarks. However, it is also reasonable to do a first round of model comparison by running models (with out of the box parameter settings) on the training data and eliminating some models which performed very poorly.

Let the best model win!

In [ ]:
def expected_score(model, x_test, y_test):
    overall = 0
    class_0 = 0
    class_1 = 0
    for i in range(100):
        sample = np.random.choice(len(x_test), len(x_test))
        x_sub_test = x_test[sample]
        y_sub_test = y_test[sample]
        
        overall += model.score(x_sub_test, y_sub_test)
        class_0 += model.score(x_sub_test[y_sub_test==0], y_sub_test[y_sub_test==0])
        class_1 += model.score(x_sub_test[y_sub_test==1], y_sub_test[y_sub_test==1])

    return pd.Series([overall / 100., 
                      class_0 / 100.,
                      class_1 / 100.],
                      index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])], 
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

In [ ]:
### fancy models that solve the problem

# Let's start with some simple classifiers to see how they do on the train and test data
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
abc = AdaBoostClassifier()
gnb = GaussianNB()
gbc = GradientBoostingClassifier()
bgc = BaggingClassifier()
knc = KNeighborsClassifier()

# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled.ravel(), test_size=0.2)

models = [dtc, rfc, abc, gnb, gbc, bgc, knc]

for model in models:
    # print("Training model", model)
    model.fit(X_resampled, y_resampled.ravel())
    print("Calculating expected score")
    print(expected_score(model, X_test_0, y_test_0))
    
    
print("done")
model = gnb    

We see that all these 'simple' classifiers without any parameters set all perform very bad on the 'flu' class. Here we also see how important it is to not only look at accuracy because they all score around 90% overall accuracy, which is totally not representative how the model performs

In [ ]:
from tensorflow import keras as keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow import keras
from keras.optimizers import SGD
import tensorflow as tf
from scipy import stats
import datetime

neuromodel = keras.Sequential([
    keras.layers.Dense(10, input_dim=X_resampled.shape[1], activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(5, activation='relu'),
#     keras.layers.Dense(10, activation='relu'),
#     keras.layers.Dense(5, activation='relu'),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

neuromodel.compile(loss='binary_crossentropy', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled.ravel(), test_size=0.2)

# Load the TensorBoard notebook extension
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = neuromodel.fit(x=X_train,
                         y=y_train,
                         epochs=15,
#                          batch_size=32,
                         validation_data=(X_test, y_test),
                         callbacks=[tensorboard_callback],
                         verbose=1)


In [ ]:
%tensorboard --logdir logs/fit

## On evaluation

### AUROC

As mentioned abbove, we will use the accuracy scores for each class and for the whole dataset, as well as the AUROC score from Kaggle platform. You can coimpute AUROC locally (e.g. on your train/validation set) by calling the relevant scikit learn function:

In [ ]:
###AUROC locally

#score = roc_auc_score(real_labels, predicted_labels)

#real_labels: the ground truth (0 or 1)
#predicted_labels: labels predicted by your algorithm (0 or 1)

### Accuracy (per class)

Below there is a function that will be handy for your models. It computes the accuracy per-class, based on a model you pass as parameter and a dataset (split to x/y)

In [ ]:
def extended_score(model, x_test, y_test):
    overall = 0
    class_0 = 0
    class_1 = 0
    for i in range(100):
        sample = np.random.choice(len(x_test), len(x_test))
        x_sub_test = x_test[sample]
        y_sub_test = y_test[sample]
        
        overall += model.score(x_sub_test, y_sub_test)
        class_0 += model.score(x_sub_test[y_sub_test==0], y_sub_test[y_sub_test==0])
        class_1 += model.score(x_sub_test[y_sub_test==1], y_sub_test[y_sub_test==1])

    return pd.Series([overall / 100., 
                      class_0 / 100.,
                      class_1 / 100.],
                      index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

In [ ]:
#same job as before, but faster?

score = lambda model, x_val, y_val: pd.Series([model.score(x_val, y_val), 
                                                 model.score(x_val[y_val==0], y_val[y_val==0]),
                                                 model.score(x_val[y_val==1], y_val[y_val==1])], 
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

## Solution extraction for Kaggle

Make sure that you extract your solutions (predictions) in the correct format required by Kaggle

In [ ]:
df_test = df_test[used_columns[:-1]]

df_test['Gender'].replace(['female','male'],[0,1],inplace=True)
df_test['HardDrugs'].fillna('No')
df_test['HardDrugs'].replace(['No','Yes'],[0,1],inplace=True)

df_test['Diabetes'].fillna('No')
df_test['Diabetes'].replace(['No','Yes'],[0,1],inplace=True)

df_test['Alcohol12PlusYr'].fillna('Yes')
df_test['Alcohol12PlusYr'].replace(['No','Yes'],[0,1],inplace=True)

df_test['SexEver'].fillna('Yes')
df_test['SexEver'].replace(['No','Yes'],[0,1],inplace=True)

df_test['SameSex'].fillna('No')
df_test['SameSex'].replace(['No','Yes'],[0,1],inplace=True)

dummies = ['Race1', 
           'Education', 
           'MaritalStatus', 
           'HomeOwn', 
           'BMI_WHO', 
           'HealthGen', 
           'LittleInterest', 
           'Depressed', 
           'TVHrsDay', 
           'CompHrsDay', 
           'SexOrientation'] 

for column in dummies:
    df_test = pd.concat([df_test, pd.get_dummies(df_test[column])], axis=1)
    
df_test.drop(dummies, axis=1)
    

df_test

In [ ]:
X = np.array(df_test.fillna(0))
predictions = neuromodel.predict(X)

file = open('predictions.csv', 'w')
with file:

    writer = csv.writer(file)
    writer.writerow(['ID', 'Prediction'])
    for i in range(len(predictions)):

        writer.writerow([df_test.iloc[i]['ID'].astype(int), int(np.round(predictions[i]))])
        

## Step 3: Conclusions

Highlight at the end of your notebook, which were the top-3 approaches that produced the best scores for you. That is, provide a table with the scores you got (on the AUROC score you get from Kaggle) and make sure that you judge these in relation to your work on the training set